In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web
import statsmodels.tsa.stattools as ts
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import pandas_market_calendars as mcal

In [2]:
end = datetime.today()
#start = end - relativedelta(years=2) - relativedelta(days=3)
start = end - relativedelta(months=64)

In [3]:
print(end.strftime('%Y-%m-%d'))
print(start.strftime('%Y-%m-%d'))

2022-06-15
2017-02-15


In [4]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1250881600Q",\
"LEU0250918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [5]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [6]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [7]:
len(sp500)+len(sp600)+len(sp400)

1505

In [8]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

504
601
400


In [9]:
#>.5The time series is mean reverting
#=.5 time series is a Geometric Brownian Motion
#<.5 time series is trending

def hurst(ts):
    """
    Returns the Hurst Exponent of the time series vector ts

    Parameters
    ----------
    ts : `numpy.array`
        Time series upon which the Hurst Exponent will be calculated

    Returns
    -------
    'float'
        The Hurst Exponent from the poly fit output
    """
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

In [10]:
nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start, end_date=end)


In [11]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

504
601
400


In [12]:
len(sp1500_index_df)-len(SP1500)

0

In [13]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    
    temp = web.DataReader(str(name), 'fred', start_, end_)
    #temp = web.DataReader(npa[1][0],'fred',start,end)
    temp = temp.asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_.index)
    temp.columns = ['Value']
    #temp.index = pd.to_datetime(temp.index)
    
    #temp['Symbol'] = name
    
    #print(temp)
    #temp_new = temp.asfreq('D').interpolate(method='time')    
    temp_dates = pd.date_range(temp.index[0].strftime('%Y-%m-%d'), temp.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp.reindex(nyse_inbetween_temp_dates)
    temp_new['Symbol'] = name
    
    #temp = temp.resample(frequency).mean().dropna()
    return(temp_new)
    #return(temp)

In [14]:

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    
    temp_new = subset.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(subset.index[0].strftime('%Y-%m-%d'), subset.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    subset_ = temp_new
    subset_['Symbol'] = name

    return (subset_)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [15]:
def getStock(npa):
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    
    data_ = yf.download(symbol, start=start_,end=end_)
  
    return([symbol,data_])

In [16]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [17]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    
    if 'sector' in tickerdata.info:
        sector = tickerdata.info['sector']
    else:
        sector = 'error'

    if 'industry' in tickerdata.info:
        industry = tickerdata.info['industry']
    else:
        industry = 'error'

    return([symbol,sector,industry])

In [18]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    nyse_dates_ = npa_[2]
    
    temp_new = data.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(data.index[0].strftime('%Y-%m-%d'), data.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    
    data_ = temp_new
    data_['Symbol'] = name

    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['MIN_Lookback_One_Year'] = data_['trailing_1yr_max'].rolling(252).min()
    #data_['MIN_Lookback_6_Months'] = data_['trailing_1yr_max'].rolling(126).min()
    data_['MIN_Lookback_2_Months'] = data_['trailing_1yr_max'].rolling(42).min()
    data_['MIN_Lookback_1_Months'] = data_['trailing_1yr_max'].rolling(21).min()

    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(252).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(252).min()
    
    #data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(126).min()
    
    #data_['supply_trend_90d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()
    
    data_['90dSMA'] = (pd.DataFrame(data_)['Adj Close']).rolling(63).mean()
    
    data_['90dSDev'] = (pd.DataFrame(data_)['Adj Close']).rolling(63 ).std()
    
    data_['TP'] = (data_['Adj Close'] + data_['Low'] + data_['High'])/3
    data_['21dstdTP'] = data_['TP'].rolling(21).std(ddof=0)
    data_['21dMA-TP'] = data_['TP'].rolling(21).mean()
    data_['21dBOLU'] = data_['21dMA-TP'] + 2*data_['21dstdTP']
    data_['21dBOLD'] = data_['21dMA-TP'] - 2*data_['21dstdTP']    

    data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_40d_max']-data_['trailing_risk_40d_min']
    
    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
    data_['30d_vol_2yr'] = data_['30d_vol'].rolling(512).sum()
    data_['volume_factor'] = data_['30d_vol']/data_['30d_vol_2yr']    
    
    #data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_6_Months']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    
    data_['risk_trend_factor'] = data_['1YMF']/data_['risk_per_share']
    
    return(data_)

In [19]:
trackers = ['beta','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInstitutions','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']

def extractTickerValues(npa):

    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    print(n_)

    subset = pd.DataFrame([r_.info])

    others = ['quarterly_balance_sheet','quarterly_cashflow','quarterly_dividends','quarterly_earnings','quarterly_financials','mutualfund_holders','options','sustainability']

    o_items = []

    #print(r_)
    for o in others:
        if(np.sum(np.array(dir(r_))==[o])>0):
            o_item = getattr(r_, o)
            o_items.append([o,o_item])            

    
    values = pd.DataFrame()

    for t in trackers:
        if(np.sum(np.where(subset.columns==t))==0):
            values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
        else:
            values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)

    return([n_,values,o_items])

In [20]:
def calculateMetrics(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Adj Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Adj Close'][[-1]].index[0] - relativedelta(years=1)
    
    #separate subset for past year.
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = pd.DataFrame(data_df)['High'].rolling(504).max()[-1]
    min_2y = pd.DataFrame(data_df)['Low'].rolling(504).min()[-1]
    
    #quantiles_close_2y = data_df['Adj Close'].rolling(504).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_2y = data_df['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_2y = pd.DataFrame(data_df)['Adj Close'].rolling(504).std()[-1]
    mean_2y = pd.DataFrame(data_df)['Adj Close'].rolling(504).mean()[-1]
    
    max_1y = past_year['High'].rolling(252).max()[-1]
    min_1y = past_year['Low'].rolling(252).min()[-1]
    
    #quantiles_close_1y = data_df['Adj Close'].rolling(252).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_1y = past_year['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_1y = past_year['Adj Close'].rolling(252).std()[-1]
    mean_1y = past_year['Adj Close'].rolling(252).mean()[-1]
    
    risk_trend_factor = data_df['risk_trend_factor'][-1]
    
    supply_trend_1yr = data_df['supply_trend_1yr'][-1]
    #supply_trend_180d = data_df['supply_trend_180d'][-1]
    #supply_trend_90d = data_df['supply_trend_90d'][-1]
    supply_trend_60d = data_df['supply_trend_60d'][-1]
    supply_trend_30d = data_df['supply_trend_30d'][-1]
    
    MIN_Lookback_One_Year = data_df['MIN_Lookback_One_Year'][-1]
    #MIN_Lookback_6_Months = data_df['MIN_Lookback_6_Months'][-1]
    MIN_Lookback_2_Months = data_df['MIN_Lookback_2_Months'][-1]
    MIN_Lookback_1_Months = data_df['MIN_Lookback_1_Months'][-1]
    
    adf = ts.adfuller(data_df['Adj Close'].values)[1]
    
    hurst_ = hurst(data_df['Adj Close'].values)

    volume_factor = data_df['volume_factor'][-1]
    vol_30d = data_df['30d_vol'][-1]
    vol_30d_2yr = data_df['30d_vol_2yr'][-1]
    
    pct_2yr = (latest_p - min_2y)/(max_2y-min_2y)
    pct_1yr = (latest_p - min_1y)/(max_1y-min_1y)
    
    ymf = data_df['1YMF'][-1]
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend_factor,supply_trend_1yr,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year, MIN_Lookback_2_Months, MIN_Lookback_1_Months,vol_30d,volume_factor,adf,hurst_,ymf])

In [21]:
#test
"""
dates = []
for d in pd.read_csv('sample.csv')['Date']:
    dates.append(datetime.strptime(d, '%d-%b-%y'))

temp = pd.read_csv('sample.csv',thousands=",")
temp.index = dates
temp = temp.iloc[:,1:]
temp.columns = ['Open','High','Low','Adj Close','Volume']
#temp['Close'] = temp['Close'].astype(float)
temp.sort_index(inplace=True)
temp

derive_price_supply_trends(['test',temp]).sort_index(ascending=False)
calculateMetrics(temp)
"""

'\ndates = []\nfor d in pd.read_csv(\'sample.csv\')[\'Date\']:\n    dates.append(datetime.strptime(d, \'%d-%b-%y\'))\n\ntemp = pd.read_csv(\'sample.csv\',thousands=",")\ntemp.index = dates\ntemp = temp.iloc[:,1:]\ntemp.columns = [\'Open\',\'High\',\'Low\',\'Adj Close\',\'Volume\']\n#temp[\'Close\'] = temp[\'Close\'].astype(float)\ntemp.sort_index(inplace=True)\ntemp\n\nderive_price_supply_trends([\'test\',temp]).sort_index(ascending=False)\ncalculateMetrics(temp)\n'

In [22]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [23]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i]])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
client.restart()
client.close()


In [24]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '77002', 'sector': 'Energy', 'fullTimeEmployees': 2430, 'longBusinessSummary': 'Targa Resources Corp., together with its subsidiary, Targa Resources Partners LP, owns, operates, acquires, and develops a portfolio of midstream energy assets in North America. It operates in two segments, Gathering and Processing, and Logistics and Transportation. The company engages in gathering, compressing, treating, processing, transporting, and selling natural gas; storing, fractionating, treating, transporting, and selling natural gas liquids (NGL) and NGL products, including services to liquefied petroleum gas exporters; and gathering, purchasing, storing, terminaling, and selling crude oil. It is also involved in the purchase and resale of NGL products; and wholesale of propane, as well as provision of related logistics services to multi-state retailers, independent retailers, and other end-users. In addition, the company offers NGL balancing services; and transportation services to refine

In [25]:
results_ticker_data[0][1].info

{'zip': '77002',
 'sector': 'Energy',
 'fullTimeEmployees': 2430,
 'longBusinessSummary': 'Targa Resources Corp., together with its subsidiary, Targa Resources Partners LP, owns, operates, acquires, and develops a portfolio of midstream energy assets in North America. It operates in two segments, Gathering and Processing, and Logistics and Transportation. The company engages in gathering, compressing, treating, processing, transporting, and selling natural gas; storing, fractionating, treating, transporting, and selling natural gas liquids (NGL) and NGL products, including services to liquefied petroleum gas exporters; and gathering, purchasing, storing, terminaling, and selling crude oil. It is also involved in the purchase and resale of NGL products; and wholesale of propane, as well as provision of related logistics services to multi-state retailers, independent retailers, and other end-users. In addition, the company offers NGL balancing services; and transportation services to ref

In [26]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [27]:

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.restart()
client.close()



C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [28]:
import pickle

ticker_values = pd.DataFrame()

pickle_dumps = []

for r in results_ticker_values:
    if(r=='error'):
        pass
        #pd.DataFrame(np.repeat('error', len(trackers), axis = None),index=)
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        pickle_dumps.append([r[0],r[2]])
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)
        
pickle.dump(pickle_dumps, open('fundamental.pkl', 'wb'))

In [29]:
market_data = pd.concat([ticker_values])

In [30]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1],nyse_dates])
    
    
prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result())
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

In [31]:
npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)

screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()

missing_screener_metrics = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = np.array([r_,*np.repeat('missing',len(pd.DataFrame(screenerMetrics).set_index(0).iloc[[0]].columns))])
    screenerMetrics.append(test)

pd.DataFrame(screenerMetrics)
        

['JW-A']


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,BANC,22.09,9.0,3.453314,16.717666,22.09,15.4,1.377088,18.918397,17.42,...,0.301943,0.643239,21.4,22.09,22.09,5916900.0,0.002042,0.202307,0.460768,-13.32
1,SLM,21.4,6.48,4.563965,15.516783,21.4,15.34,1.090922,18.513413,16.99,...,0.272277,0.704424,20.82,21.4,21.4,58839000.0,0.001452,0.751209,0.499331,-12.65
2,CENX,30.360001,6.26,4.840968,13.702936,30.360001,10.13,4.884908,16.08123,10.75,...,0.030648,0.186307,19.6,30.360001,30.360001,48928500.0,0.00301,0.198719,0.397017,-48.070002
3,NBR,207.669998,21.66,40.271542,89.811012,207.669998,65.580002,31.405016,115.630516,153.380005,...,0.617918,0.708134,133.610001,195.880005,207.669998,3499000.0,0.000989,0.001083,0.496042,-77.019989
4,CME,256.940002,146.889999,25.951692,199.537973,256.940002,185.789993,16.125333,217.676111,196.110001,...,0.145046,0.447251,221.820007,256.940002,256.940002,35748200.0,0.002128,0.387037,0.387045,-147.37001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,IRDM,54.650002,23.32,6.099904,37.067123,48.950001,31.73,3.036057,39.528056,34.970001,...,0.188153,0.371848,48.950001,48.950001,48.950001,9219200.0,0.000967,0.456237,0.38411,-41.939999
1516,TSN,100.720001,55.279999,12.010072,74.264111,100.720001,69.879997,7.817635,83.491868,83.919998,...,0.455253,0.630282,81.790001,100.720001,100.720001,44566700.0,0.001836,0.378445,0.497863,-31.470009
1517,JWN,46.450001,11.72,8.304014,26.881865,38.48,18.65,4.861973,27.364762,25.870001,...,0.364095,0.407429,38.48,38.48,38.48,158446900.0,0.002262,0.404333,0.503326,-37.829996
1518,UVV,64.129997,38.82,7.095426,49.193175,64.129997,46.240002,4.692383,52.929138,56.810001,...,0.590833,0.710786,60.950001,60.950001,61.709999,2439400.0,0.001694,0.222747,0.429082,-13.179996


In [32]:
metrics = pd.concat([pd.DataFrame(screenerMetrics)],axis=0)

#return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend_factor,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,,pct_2yr])
#return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend_factor,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year, MIN_Lookback_6_Months, MIN_Lookback_2_Months, MIN_Lookback_1_Months,vol_30d,vol_30d_2yr,volume_factor,adf,hurst_])
metrics.columns = ['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct','risk_trend_factor','supply_trend_1yr','supply_trend_60d','supply_trend_30d','vol_30d_2yr','pct_1yr','pct_2yr','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','vol_30d','volume_factor','adf','hurst','1YMF']

screener = metrics
a = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend_factor'].apply(type) != np.str_].sort_values(by='risk_trend_factor',ascending=False)
b = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend_factor'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)


In [33]:
#set(np.array(market_data.index)).difference(pd.DataFrame(screenerMetrics).set_index(0).index)

In [34]:

npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])
    


In [35]:

completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=completed_fred.index,columns=['Symbol'])
completed_fred_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
2017-02-15   NaN     NaN             NaN   2.19               NaN   
2017-02-16   NaN     NaN             NaN   2.20               NaN   
2017-02-17   NaN     NaN             NaN   2.21               NaN   
2017-02-21   NaN     NaN             NaN   2.20               NaN   
2017-02-22   NaN     NaN             NaN   2.21               NaN   
...          ...     ...             ...    ...               ...   
2022-06-09   NaN     NaN             NaN   2.07               NaN   
2022-06-10   NaN     NaN             NaN   2.04               NaN   
2022-06-13   NaN     NaN             NaN   1.98               NaN   
2022-06-14   NaN     NaN             NaN   1.99               NaN   
2022-06-15   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
2017-02-15           NaN       1.23           589.78           767.28   
2017-02-16           NaN       1.24           592.46           769.93   
2017-02-17           NaN       1.24           593.53           771.08   
2017-02-21           NaN       1.24           593.63           771.49   
2017-02-22           NaN       1.23           593.91           772.13   
...                  ...        ...              ...              ...   
2022-06-09           NaN       1.38           665.22           874.71   
2022-06-10           NaN       1.41           657.99           865.51   
2022-06-13           NaN       1.46           643.33           849.72   
2022-06-14           NaN       1.46           639.08           843.98   
2022-06-15           NaN        NaN              NaN              NaN   

                        ...                                                    \
Symbol     BAMLH0A3HYC  ... USSLIND USSTHPI VIXCLS VXVCLS  WDFUELLA    WGS3MO   
2017-02-15        8.30  ...     NaN     NaN  11.97  13.41       NaN       NaN   
2017-02-16        8.32  ...     NaN     NaN  11.76  13.63       NaN       NaN   
2017-02-17        8.35  ...     NaN     NaN  11.49  13.72  1.659000  0.530000   
2017-02-21        8.28  ...     NaN     NaN  11.57  14.04  1.665286  0.524286   
2017-02-22        8.21  ...     NaN     NaN  11.74  14.39  1.666857  0.522857   
...                ...  ...     ...     ...    ...    ...       ...       ...   
2022-06-09        9.96  ...     NaN     NaN  26.09  28.01  4.561429  1.281429   
2022-06-10       10.11  ...     NaN     NaN  27.75  29.72  4.563000  1.300000   
2022-06-13       10.55  ...     NaN     NaN  34.02  34.24       NaN       NaN   
2022-06-14       10.57  ...     NaN     NaN  32.69  33.28       NaN       NaN   
2022-06-15         NaN  ...     NaN     NaN    NaN    NaN       NaN       NaN   

                                                           
Symbol     WILLLRGCAPVAL WILLMICROCAPPR WPU0911    WTB3MS  
2017-02-15      71018.04       12552.90     NaN       NaN  
2017-02-16      71035.46       12503.16     NaN       NaN  
2017-02-17      71179.78       12555.10     NaN  0.530000  
2017-02-21      71622.08       12579.64     NaN  0.524286  
2017-02-22      71509.60       12530.05     NaN  0.522857  
...                  ...            ...     ...       ...  
2022-06-09     119772.15       13844.43     NaN  1.242857  
2022-06-10     117117.70       13492.97     NaN  1.260000  
2022-06-13     113099.53       12761.72     NaN       NaN  
2022-06-14     112315.90       12673.79     NaN       NaN  
2022-06-15           NaN            NaN     NaN       NaN  

[1343 rows x 135 columns]

In [36]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me resultsa
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=completed_bonds.index,columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
2017-02-15,96.740143,103.261238,87.258522,100.497971,22.746798,95.970566,113.400192,51.725052,43.281620,44.499977,43.773109,101.977051,47.439785,95.970039,45.893383,104.686028,79.635895,99.252007,106.496597
2017-02-16,96.946388,103.454025,87.258522,100.722130,22.801611,96.356331,113.715279,51.742035,43.299351,44.553288,43.860313,102.264389,47.372372,96.210213,45.893383,104.676537,79.673653,99.317947,107.078514
2017-02-17,97.161591,103.600906,87.277618,100.937218,22.847296,96.613533,113.909920,51.937355,43.441082,44.588844,43.860313,102.525635,47.505280,96.334740,45.902515,104.695488,79.730247,99.355621,107.714104
2017-02-21,97.152618,103.591713,87.258522,100.892441,22.856426,96.567581,113.854317,51.996784,43.414505,44.588844,43.851589,102.569176,47.302143,96.334740,45.902515,104.704979,79.739685,99.393250,107.526100
2017-02-22,97.215393,103.692703,87.258522,101.053749,22.865564,96.723694,113.956268,52.141167,43.444630,44.597736,43.947502,102.647530,47.302143,96.468185,45.911644,104.695488,79.758530,99.440361,107.705124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-08,102.160004,109.959999,91.430000,108.139999,23.889999,101.820000,119.150002,56.500000,45.160000,47.689999,47.240002,112.339996,49.770000,107.360001,49.430000,110.160004,82.959999,104.820000,114.099998
2022-06-09,102.050003,109.769997,91.440002,107.830002,23.889999,101.730003,119.010002,56.320000,45.119999,47.599998,47.180000,111.599998,49.775002,106.940002,49.400002,110.150002,82.900002,104.870003,114.480003
2022-06-10,101.260002,109.190002,91.419998,107.099998,23.719999,100.730003,118.040001,55.830002,44.610001,47.320000,46.770000,110.519997,49.740002,106.559998,49.380001,110.099998,82.550003,104.709999,113.769997
2022-06-13,99.599998,107.860001,91.430000,105.269997,23.400000,98.940002,116.720001,54.110001,43.748001,46.860001,45.799999,107.970001,49.650002,105.199997,49.279999,110.010002,82.129997,103.930000,110.199997


In [37]:
completed_bonds

,Open,High,Low,Close,Volume,Symbol
2017-02-15,45.884239,45.911668,45.875098,45.893383,2303200.0,NEAR
2017-02-16,45.902524,45.902524,45.884239,45.893383,856900.0,NEAR
2017-02-17,45.884231,45.911659,45.884231,45.902515,588000.0,NEAR
2017-02-21,45.893375,45.902515,45.893375,45.902515,244700.0,NEAR
2017-02-22,45.893359,45.920785,45.893359,45.911644,249200.0,NEAR
...,...,...,...,...,...,...
2022-06-08,114.680000,115.129997,114.080002,114.099998,11476100.0,TLT
2022-06-09,113.839996,114.709999,113.680000,114.480003,15527300.0,TLT
2022-06-10,114.360001,114.610001,112.970001,113.769997,21912300.0,TLT
2022-06-13,111.489998,111.669998,109.239998,110.199997,36968300.0,TLT


In [38]:
screener_sorted

,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,2yr_close_0pct,...,pct_1yr,pct_2yr,MIN_Lookback_One_Year,MIN_Lookback_2_Months,MIN_Lookback_1_Months,vol_30d,volume_factor,adf,hurst,1YMF
Symbol,,,,,,,,,,,,,,,,,,,,,
ACC,65.220001,31.07,9.278184,45.979006,65.220001,46.720001,5.615674,53.64705,64.419998,18.98494,...,0.956757,0.976574,49.209999,58.66,65.019997,60845900.0,0.004415,0.979637,0.418255,20.369999
MTOR,36.299999,18.16,4.641666,26.996905,36.299999,20.5,5.486852,27.9325,36.0,12.11,...,0.981013,0.983462,32.93,36.240002,36.279999,24714600.0,0.003133,0.304331,0.434725,2.549999
Y,862.869995,460.579987,90.400146,650.790456,862.869995,585.099976,76.53622,706.084247,832.0,451.859985,...,0.888865,0.923264,737.890015,862.869995,862.869995,2396700.0,0.002892,0.38518,0.458449,32.369995
TWI,19.66,1.22,4.13573,8.058472,19.66,6.69,3.480319,10.486984,19.15,1.1,...,0.960678,0.972343,11.82,15.77,17.6,13793300.0,0.002554,0.891295,0.499041,12.259998
PBF,44.119999,4.06,7.198174,14.031984,44.119999,7.24,7.825139,17.745913,38.919998,4.27,...,0.859002,0.870195,18.780001,27.690001,32.41,68490300.0,0.001441,0.699548,0.555964,37.879993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GXO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.189999,48.169998,...,NaN,NaN,NaN,NaN,NaN,24785600.0,NaN,0.711826,0.476323,NaN
CCSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.060001,35.630001,...,NaN,NaN,NaN,NaN,NaN,3297200.0,NaN,0.033363,0.1202,NaN
AIRC,NaN,NaN,NaN,NaN,55.82,39.119999,3.114302,50.807738,39.439999,36.15271,...,0.019162,NaN,NaN,55.529999,55.82,20033100.0,NaN,0.441101,0.406442,NaN


In [39]:
#writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

data_object = [start,end,prices_df, sp1500_index_df, sp500, sp600, sp400, market_data, completed_fred_pvt, completed_bonds, completed_bonds_pvt, pd.DataFrame(sectors,sector_names), pd.DataFrame(indexes,index_names), screener_sorted]
               
pickle.dump(data_object, open('data_object.pkl', 'wb'))
"""
#prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

#sectors_df.to_excel(writer, sheet_name='stock_sectors')

#mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_sorted.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
wb.close()
writer.close()
"""
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"